In [1]:
import random
import numpy as np
import networkx as nx
import networkit as nk
from typing import Union
from collections import deque
from littleballoffur.sampler import Sampler
import torch
import torch_geometric.transforms as T
from torch_geometric.datasets.dblp import DBLP
from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx

NKGraph = type(nk.graph.Graph())
NXGraph = nx.classes.graph.Graph

Small graphs are obtained from real datasets by modifying the ForestFireSampler code in https://little-ball-of-fur.readthedocs.io/en/latest/_modules/littleballoffur/exploration_sampling/forestfiresampler.html

In [2]:
dataset_name = 'dblp'
nodetype = 1
classes = [0,1,2,3]
feature_size = 50

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import csv
import random
import os

from littleballoffur import ForestFireSampler
import time

In [4]:
class ForestFireSampler(Sampler):
    r"""An implementation of forest fire sampling. The procedure is a stochastic
    snowball sampling method where the expansion is proportional to the burning probability.
    `"For details about the algorithm see this paper." <https://cs.stanford.edu/people/jure/pubs/sampling-kdd06.pdf>`_


    Args:
        number_of_nodes (int): Number of sampled nodes. Default is 100.
        p (float): Burning probability. Default is 0.4.
        seed (int): Random seed. Default is 42.
    """

    def __init__(
            self,
            number_of_nodes: int = 100,
            p: float = 0.4,
            seed: int = 42,
            max_visited_nodes_backlog: int = 100,
            restart_hop_size: int = 10,
    ):
        self.number_of_nodes = number_of_nodes
        self.p = p
        #self.seed = seed
        #self._set_seed()
        self.restart_hop_size = restart_hop_size
        self.max_visited_nodes_backlog = max_visited_nodes_backlog

    def _create_node_sets(self, graph):
        """
        Create a starting set of nodes.
        """
        self._sampled_nodes = set()
        self._set_of_nodes = set(range(self.backend.get_number_of_nodes(graph)))
        self._visited_nodes = deque(maxlen=self.max_visited_nodes_backlog)

    def _start_a_fire(self, graph):
        """
        Starting a forest fire from a single node.
        """
        remaining_nodes = list(self._set_of_nodes.difference(self._sampled_nodes))
        #seed_node = random.choice(remaining_nodes)
        #modified code to select start node from the list of Author nodes
        author_nodes= [x for x, y in undirected_graph.nodes(data=True) if y['type'] == 1]
        seed_node = random.choice(author_nodes)
        self._sampled_nodes.add(seed_node)
        node_queue = deque([seed_node])
        while len(self._sampled_nodes) < self.number_of_nodes:
            if len(node_queue) == 0:
                node_queue = deque(
                    [
                        self._visited_nodes.popleft()
                        for k in range(
                        min(self.restart_hop_size, len(self._visited_nodes))
                    )
                    ]
                )
                if len(node_queue) == 0:
                    print(
                        "Warning: could not collect the required number of nodes. The fire could not find enough nodes to burn."
                    )
                    break
            top_node = node_queue.popleft()
            self._sampled_nodes.add(top_node)
            neighbors = set(self.backend.get_neighbors(graph, top_node))
            unvisited_neighbors = neighbors.difference(self._sampled_nodes)
            score = np.random.geometric(self.p)
            count = min(len(unvisited_neighbors), score)
            burned_neighbors = random.sample(unvisited_neighbors, count)
            self._visited_nodes.extendleft(
                unvisited_neighbors.difference(set(burned_neighbors))
            )
            for neighbor in burned_neighbors:
                if len(self._sampled_nodes) >= self.number_of_nodes:
                    break
                node_queue.extend([neighbor])

    def sample(self, graph: Union[NXGraph, NKGraph]) -> Union[NXGraph, NKGraph]:
        """
        Sampling nodes iteratively with a forest fire sampler.

        Arg types:
            * **graph** *(NetworkX or NetworKit graph)* - The graph to be sampled from.

        Return types:
            * **new_graph** *(NetworkX or NetworKit graph)* - The graph of sampled nodes.
        """
        self._deploy_backend(graph)
        self._check_number_of_nodes(graph)
        self._create_node_sets(graph)
        while len(self._sampled_nodes) < self.number_of_nodes:
            self._start_a_fire(graph)
        new_graph = self.backend.get_subgraph(graph, self._sampled_nodes)
        return new_graph

In [5]:
def plot_graph(G,attribute):

    color_class_map = {0: 'blue', 1: 'red', 2: 'darkgreen', 3: 'orange'}

    nx.draw(G, 
        with_labels=True, node_color=[color_class_map[node[1][attribute]] 
                        for node in G.nodes(data=True)], 
            node_size=200,
        font_color='black')
    plt.show() 

DBLP

In [7]:
def get_node_types(G):
    
    for n in G.nodes:
        node_type = G.nodes[n]['type']
        if node_type == 'author':
            G.nodes[n]['type'] = 1
        elif node_type == 'paper':
            G.nodes[n]['type'] = 0
        elif node_type == 'term':
            G.nodes[n]['type'] = 2
        elif node_type == 'conference':
            G.nodes[n]['type'] = 3

In [8]:
dataset = DBLP(root='./dblp_data', transform=T.Constant(node_types='conference'))
data = dataset[0]

In [9]:
author = data['author'].x.tolist()
author_df = pd.DataFrame(data['author'].x)
author_df['class'] = data['author'].y.tolist()

In [10]:
def feature_selection_var(X,threshold=0.0):
    sel = VarianceThreshold(threshold=(threshold * (1 - threshold)))
    fitted_X = sel.fit_transform(X)
    imp_feat = pd.DataFrame(fitted_X)
    
    return imp_feat

In [12]:
networkX_graph = to_networkx(data, node_attrs=['x'])
undirected_graph = networkX_graph.to_undirected()
get_node_types(undirected_graph)

In [14]:
#Feature selection for author and paper nodes
author_features=[]
paper_features=[]
for n in undirected_graph.nodes:
    node_type = undirected_graph.nodes[n]['type']
    if node_type == 1:
        author_features.append(undirected_graph.nodes[n]['x'])
        
    elif node_type == 0:
        paper_features.append(undirected_graph.nodes[n]['x'])

selected_author_feat = feature_selection_var(pd.DataFrame(author_features),threshold=0.948).iloc[:, : feature_size]
selected_paper_feat = feature_selection_var(pd.DataFrame(paper_features),threshold=0.9845).iloc[:, : feature_size]

In [15]:
#Add selected features to nodes and one-hot encoding of node types to features
author_node_counter=0
paper_node_counter=0

ohe_paper = list(np.eye(len(classes))[1])
ohe_author = list(np.eye(len(classes))[0])
ohe_term = list(np.eye(len(classes))[2])
ohe_conference = list(np.eye(len(classes))[3])

for n in undirected_graph.nodes:
    node_type = undirected_graph.nodes[n]['type']
    if node_type == 1:  
        author_feat = selected_author_feat.iloc[author_node_counter].to_list()
        author_feat.extend(ohe_author)
        undirected_graph.nodes[n]['x'] = author_feat
        author_node_counter = author_node_counter + 1
        
    elif node_type == 0:    
        paper_feat = selected_paper_feat.iloc[paper_node_counter].to_list()
        paper_feat.extend(ohe_paper)
        undirected_graph.nodes[n]['x'] = paper_feat
        paper_node_counter = paper_node_counter + 1

    elif node_type == 2:    
        term_feat = undirected_graph.nodes[n]['x']
        term_feat.extend(ohe_term)
        undirected_graph.nodes[n]['x'] = term_feat
        
    elif node_type == 3:   
        conf_feat = list(np.zeros(50))
        conf_feat.extend(ohe_conference)
        undirected_graph.nodes[n]['x'] = conf_feat
    


In [19]:
len(undirected_graph.nodes(data=True)[0]['x'])

54

In [20]:
for nodesize in range(10,16):
    adj_list= []
    node_type_list = []
    node_feature_list = []
    for i in range(0,200):
        model = ForestFireSampler(nodesize)
        graph = model.sample(undirected_graph)
        if nx.is_connected(graph):

            A = nx.adjacency_matrix(graph)
            adj = torch.tensor(np.asarray(A.todense()))
            adj_list.append(adj)
            
            node_type = nx.get_node_attributes(graph, "type")
            node_feature = nx.get_node_attributes(graph, "x")
            
            node_type_list.append(list(node_type.values()))
            node_feature_list.append(list(node_feature.values()))

        else:
            print('Disconnected graph')  
        torch.save([adj_list,node_type_list,node_feature_list],'baseline_vae/real_' + dataset_name+ '_vae_var/'+ str(nodesize) + '.pt')

In [21]:
len(adj_list)

200

In [22]:
adjs, node_types, node_feats = torch.load('baseline_vae/real_dblp_vae_var/10.pt')

In [23]:
adjs[0]

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
        [1, 1, 0, 1, 0, 1, 1, 0, 1, 0]])

In [29]:
node_types[0]

[2, 1, 0, 2, 0, 2, 1, 0, 1, 0]

In [33]:
node_feats[0][0]

[0.16902999579906464,
 1.2401000261306763,
 0.06723000109195709,
 -0.389629989862442,
 0.3119100034236908,
 1.1172000169754028,
 0.7573300004005432,
 -0.8892999887466431,
 0.6833599805831909,
 -0.753600001335144,
 -0.1707800030708313,
 -0.3740200102329254,
 -0.02481199987232685,
 -0.015793999657034874,
 0.17962999641895294,
 -0.0433569997549057,
 -0.163100004196167,
 -0.061482999473810196,
 -0.8431199789047241,
 -0.8103700280189514,
 -0.36487001180648804,
 -0.3984299898147583,
 0.9004499912261963,
 -0.041652001440525055,
 0.382860004901886,
 -0.4183900058269501,
 -0.07397300004959106,
 1.1279000043869019,
 0.28856000304222107,
 -0.1861100047826767,
 3.1709001064300537,
 -0.26805999875068665,
 0.5761299729347229,
 0.6187499761581421,
 0.2998400032520294,
 -0.43720999360084534,
 0.6200199723243713,
 0.37286999821662903,
 -0.7694100141525269,
 0.05594500154256821,
 0.18544000387191772,
 0.02395699918270111,
 0.20765000581741333,
 0.3058300018310547,
 -1.4075000286102295,
 -0.8818500041961

In [32]:
len(node_feats[0][0])

54